<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jaiio/00-04-static_features_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de las features adicionales a la Consulta


## Features estáticas a partir de los datos

1. Traigo los datos y los cargo en el dataframe:

In [1]:
from os import path
import pandas as pd

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists("dataset-tfidf.csv")):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/50jaiio/consolidados/feature-extraction/dataset-tfidf.csv

df = pd.read_csv('dataset-tfidf.csv')

df['clase'].value_counts()


Requisitos de Ingreso                  50
Situación Académica                    50
Cambio de Comisión                     50
Consulta sobre Título Universitario    50
Cambio de Carrera                      50
Pedido de Certificados                 50
Consulta por Equivalencias             50
Simultaneidad de Carreras              50
Reincorporación                        50
Problemas con la Clave                 50
Consulta por Legajo                    50
Cursadas                               50
Boleto Universitario                   50
Exámenes                               50
Datos Personales                       50
Ingreso a la Universidad               50
Name: clase, dtype: int64

In [2]:
df.columns

Index(['fecha', 'hora', 'apellido_nombre', 'legajo', 'documento', 'carrera',
       'telefono', 'email', 'consulta', 'respuesta', 'score', 'clase'],
      dtype='object')

2. Verifico los datos y borro columnas no pre-procesables:

In [3]:
df.drop(columns=['apellido_nombre', 'respuesta'], inplace=True)
df.head()

,fecha,hora,legajo,documento,carrera,telefono,email,consulta,score,clase
0,05-03-2019,12:55:26,170301,41783126,licenciatura en administracion(3),1137722125,anlemargagliotti@gmail.com,"hola, hice hace un mes y algo el tramite onlin...",37.415314,Boleto Universitario
1,10-15-2019,04:06:31,160474,40993972,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",34.819030,Boleto Universitario
2,07-30-2018,20:20:20,165259,42344375,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,33.004665,Boleto Universitario
3,07-13-2018,10:09:06,163392,41925156,profesorado en educacion fisica(43),1536771314,danilamferreras@gmail.com,"hola que tal?, buenos días. quería consultar ...",33.004196,Boleto Universitario
4,05-21-2019,00:26:56,161066,41677207,tec. un. en insp. de alimentos(19),232315484535,barbu_99@hotmail.com,hola queria saber xque no puedo hacer el bolet...,31.860080,Boleto Universitario


3. Preproceso la fecha (día de la semana, semana del mes, mes y cuatrimestre) de la consulta:

In [4]:
def convierte_fecha(fecha_consulta):
  from datetime import datetime
  # Convierto a fecha
  fecha = datetime.strptime(fecha_consulta, '%m-%d-%Y') 
  return fecha

def dia_semana(fecha):
  # Tomo el día de la semana
  dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
  # dia_semana = dias_semana[fecha.weekday()]
  dia_semana = fecha.weekday()
  return dia_semana

def semana_del_mes(fecha):
  # Tomo la semana del mes
  semana_mes = (fecha.day-1)//7+1
  return semana_mes

def mes(fecha):
  # Tomo el mes
  meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
  # mes = meses[fecha.month-1]
  mes = fecha.month
  return mes

def cuatrimestre(fecha):
  # Defino el cuatrimestre
  cuatrimestre = 1
  if (fecha.month>7):
    cuatrimestre = 2
  return cuatrimestre

def anio(fecha):
    anio = fecha.year
    return anio

# convierte_fecha(df.loc[0, 'Fecha'])
fecha = df['fecha'].apply(convierte_fecha)
df['dia_semana'] = fecha.apply(dia_semana)
df['semana_del_mes'] = fecha.apply(semana_del_mes)
df['mes'] = fecha.apply(mes)
df['cuatrimestre'] = fecha.apply(cuatrimestre)
df['anio'] = fecha.apply(anio)

In [5]:
df.drop(columns=['fecha'], inplace=True)
df.head()

,hora,legajo,documento,carrera,telefono,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio
0,12:55:26,170301,41783126,licenciatura en administracion(3),1137722125,anlemargagliotti@gmail.com,"hola, hice hace un mes y algo el tramite onlin...",37.415314,Boleto Universitario,4,1,5,1,2019
1,04:06:31,160474,40993972,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",34.819030,Boleto Universitario,1,3,10,2,2019
2,20:20:20,165259,42344375,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,33.004665,Boleto Universitario,0,5,7,1,2018
3,10:09:06,163392,41925156,profesorado en educacion fisica(43),1536771314,danilamferreras@gmail.com,"hola que tal?, buenos días. quería consultar ...",33.004196,Boleto Universitario,4,2,7,1,2018
4,00:26:56,161066,41677207,tec. un. en insp. de alimentos(19),232315484535,barbu_99@hotmail.com,hola queria saber xque no puedo hacer el bolet...,31.860080,Boleto Universitario,1,3,5,1,2019


4. Preproceso la hora de la consulta de la siguiente forma:
    - 0-Mañana (6-12 hs),
    - 1-Media-Tarde (12-16 hs)
    - 2-Tarde (16-20 hs)
    - 3-Noche (20-00 hs)
    - 4-Madrugada (00-6 hs)

In [6]:
def convierte_horario(hora_consulta):
  from datetime import datetime
  # Convierto a hora
  hora = datetime.strptime(hora_consulta, '%H:%M:%S').time()
  return hora

def discretiza_horario(horario):
  if (horario.hour>=6 and horario.hour<12):
    rango_horario=0
  elif (horario.hour>=12 and horario.hour<16):
    rango_horario=1
  elif (horario.hour>=16 and horario.hour<20):
    rango_horario=2
  else:
    rango_horario=3 
  return rango_horario

#discretiza_horario(convierte_horario(df.loc[0, 'Hora']))
horario = df['hora'].apply(convierte_horario)
df['hora_discretizada'] = horario.apply(discretiza_horario)

In [7]:
df.drop(columns=['hora'], inplace=True)
df.head()

,legajo,documento,carrera,telefono,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada
0,170301,41783126,licenciatura en administracion(3),1137722125,anlemargagliotti@gmail.com,"hola, hice hace un mes y algo el tramite onlin...",37.415314,Boleto Universitario,4,1,5,1,2019,1
1,160474,40993972,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",34.819030,Boleto Universitario,1,3,10,2,2019,3
2,165259,42344375,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,33.004665,Boleto Universitario,0,5,7,1,2018,3
3,163392,41925156,profesorado en educacion fisica(43),1536771314,danilamferreras@gmail.com,"hola que tal?, buenos días. quería consultar ...",33.004196,Boleto Universitario,4,2,7,1,2018,0
4,161066,41677207,tec. un. en insp. de alimentos(19),232315484535,barbu_99@hotmail.com,hola queria saber xque no puedo hacer el bolet...,31.860080,Boleto Universitario,1,3,5,1,2019,3


5. Preproceso el legajo y el DNI:<br />
  Los separo en cuantiles asumiendo que los DNI mas bajos son personas mas grandes y los legajos mas grandes son estudiantes mas nuevos.

In [8]:
import numpy as np
 
def discretiza_atributo(atributo, intervalos=4):
  # Se discretiza según la cantidad de q definidos
  discretizado = pd.qcut(atributo, q=intervalos)
  
  # Nos quedamos con los diferentes intervalos, los ordenamos min-max
  lista_intervalos = discretizado.unique()

  lista_intervalos.sort_values(inplace=True)
  #  Lo casteamos a str
  lista_intervalos = lista_intervalos.astype(str)
  #  Casteamos el atributo a str
  discretizado = discretizado.astype(str)
  # Borro los nan porque los modifico a mano
  tiene_nan = False
  if ('nan' in lista_intervalos):
    tiene_nan = True
    np.delete(lista_intervalos, np.where(lista_intervalos == 'nan'))

  for i in range(intervalos):
    # Si existiera, quiero dejar el nan en el intervalo con etiqueta 0, por eso sumo 1 a i
    discretizado = discretizado.replace(lista_intervalos[i], i+1)
    discretizado = discretizado.replace('nan', 0)

  return discretizado

# Cambio a numericos los Documentos y los discretizo
df["documento"] = pd.to_numeric(df["documento"], errors='coerce')
df["dni_discretizado"] = discretiza_atributo(df["documento"], 8)

# Cambio a numericos los Legajos y los discretizo
df["legajo"] = pd.to_numeric(df["legajo"], errors='coerce')
df["legajo_discretizado"] = discretiza_atributo(df["legajo"], 4)

In [9]:
df["dni_discretizado"].value_counts()

3    101
8    100
7    100
6    100
5    100
2    100
1    100
4     99
Name: dni_discretizado, dtype: int64

In [10]:
df["legajo_discretizado"].value_counts()

0    227
1    145
4    143
3    143
2    142
Name: legajo_discretizado, dtype: int64

6. Genero una variable dummy con la existencia o no de Legajo y Teléfono:

In [11]:
# Tiene legajo?
def posee_valor(atributo):
  return 1-atributo.isna()
  
df['posee_legajo'] = posee_valor(df['legajo'])

In [12]:
df.drop(columns=['legajo', 'documento'], inplace=True)
df.head()

,carrera,telefono,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo
0,licenciatura en administracion(3),1137722125,anlemargagliotti@gmail.com,"hola, hice hace un mes y algo el tramite onlin...",37.415314,Boleto Universitario,4,1,5,1,2019,1,8,4,1
1,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",34.819030,Boleto Universitario,1,3,10,2,2019,3,7,4,1
2,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,33.004665,Boleto Universitario,0,5,7,1,2018,3,8,4,1
3,profesorado en educacion fisica(43),1536771314,danilamferreras@gmail.com,"hola que tal?, buenos días. quería consultar ...",33.004196,Boleto Universitario,4,2,7,1,2018,0,8,4,1
4,tec. un. en insp. de alimentos(19),232315484535,barbu_99@hotmail.com,hola queria saber xque no puedo hacer el bolet...,31.860080,Boleto Universitario,1,3,5,1,2019,3,7,4,1


In [13]:
# Tiene teléfono? 
df['posee_telefono'] = posee_valor(df['telefono'])

In [14]:
df.drop(columns=['telefono'], inplace=True)
df.head()

,carrera,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono
0,licenciatura en administracion(3),anlemargagliotti@gmail.com,"hola, hice hace un mes y algo el tramite onlin...",37.415314,Boleto Universitario,4,1,5,1,2019,1,8,4,1,1
1,licenciatura en administracion(3),mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",34.819030,Boleto Universitario,1,3,10,2,2019,3,7,4,1,0
2,licenciatura en administracion(3),agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,33.004665,Boleto Universitario,0,5,7,1,2018,3,8,4,1,1
3,profesorado en educacion fisica(43),danilamferreras@gmail.com,"hola que tal?, buenos días. quería consultar ...",33.004196,Boleto Universitario,4,2,7,1,2018,0,8,4,1,1
4,tec. un. en insp. de alimentos(19),barbu_99@hotmail.com,hola queria saber xque no puedo hacer el bolet...,31.860080,Boleto Universitario,1,3,5,1,2019,3,7,4,1,1


7. Se extrae el código de Carrera:

In [15]:
def extrae_codigo_carrera(texto_carrera):
  codigo = 0
  if "sin carrera" not in texto_carrera:
    texto_carrera = texto_carrera.split('(')
    texto_carrera = texto_carrera[len(texto_carrera)-1].split(')')
    codigo = int(texto_carrera[0])
  return codigo

# extrae_codigo_carrera(df.loc[0, 'Carrera'])
df['carrera_valor'] = df['carrera'].apply(extrae_codigo_carrera)

In [16]:
df.drop(columns=['carrera'], inplace=True)
df.head()

,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor
0,anlemargagliotti@gmail.com,"hola, hice hace un mes y algo el tramite onlin...",37.415314,Boleto Universitario,4,1,5,1,2019,1,8,4,1,1,3
1,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",34.819030,Boleto Universitario,1,3,10,2,2019,3,7,4,1,0,3
2,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,33.004665,Boleto Universitario,0,5,7,1,2018,3,8,4,1,1,3
3,danilamferreras@gmail.com,"hola que tal?, buenos días. quería consultar ...",33.004196,Boleto Universitario,4,2,7,1,2018,0,8,4,1,1,43
4,barbu_99@hotmail.com,hola queria saber xque no puedo hacer el bolet...,31.860080,Boleto Universitario,1,3,5,1,2019,3,7,4,1,1,19


8. Se verifica el proveedor de correo electrónico:

In [17]:
def servicio_email(consulta, proveedor):  
  if consulta.lower().find(proveedor)==-1:
    return 0
  else:
    return 1
# Correo Gmail?
# df['correo_gmail'] = df['E-mail'].apply(servicio_email, proveedor='gmail')
# Correo yahoo?
# df['correo_yahoo'] = df['E-mail'].apply(servicio_email, proveedor='yahoo')
# Correo hotmail?
# df['correo_hotmail'] = df['E-mail'].apply(servicio_email, proveedor='hotmail')

def extrae_proveedor_correo(texto_correo):
    texto_correo = texto_correo.split('@')
    texto_correo = texto_correo[len(texto_correo)-1].split('.')
    proveedor_correo = texto_correo[0]
    return proveedor_correo

#extrae_proveedor_correo(df.loc[0, 'E-mail'])
df['proveedor_correo'] = df['email'].apply(extrae_proveedor_correo)

In [18]:
df['proveedor_correo'].value_counts()

hotmail            371
gmail              308
yahoo               41
outlook             37
live                32
udesa                2
homail               1
molinoargentino      1
santanderrio         1
edenor               1
colegiodelpilar      1
uno                  1
gamail               1
hotmaill             1
icloud               1
Name: proveedor_correo, dtype: int64

In [19]:
df.drop(columns=['email'], inplace=True)
df.head()

,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo
0,"hola, hice hace un mes y algo el tramite onlin...",37.415314,Boleto Universitario,4,1,5,1,2019,1,8,4,1,1,3,gmail
1,"hola, en la página dice que tengo asignado el ...",34.819030,Boleto Universitario,1,3,10,2,2019,3,7,4,1,0,3,gmail
2,yo hace casi un mes hice el tramite para la su...,33.004665,Boleto Universitario,0,5,7,1,2018,3,8,4,1,1,3,gmail
3,"hola que tal?, buenos días. quería consultar ...",33.004196,Boleto Universitario,4,2,7,1,2018,0,8,4,1,1,43,gmail
4,hola queria saber xque no puedo hacer el bolet...,31.860080,Boleto Universitario,1,3,5,1,2019,3,7,4,1,1,19,hotmail


## Features estáticas léxicas

Genero las features estáticas léxicas basadas en caracteres:

In [24]:
# Total number of characters
def cantidad_caracteres(columna_consulta):
  return columna_consulta.str.len()

df['cantidad_caracteres'] = cantidad_caracteres(df.consulta)

# Proporcion de mayúsculas en la consulta
def proporcion_mayusculas(consulta):
  return sum(1 for letra in consulta if letra.isupper())

df['proporcion_mayusculas'] = df['consulta'].apply(proporcion_mayusculas)/df['cantidad_caracteres']

# Proporción de letras en la consulta
def proporcion_letras(consulta):
  return sum(1 for letra in consulta if letra.isalpha())

df['proporcion_letras'] = df['consulta'].apply(proporcion_letras)/df['cantidad_caracteres']

# Cantidad de letras con tildes
def cantidad_tildes(consulta):
  return sum(1 for letra in consulta if letra.lower() in ['á', 'é', 'í', 'ó', 'ú'])

df['cantidad_tildes'] = df['consulta'].apply(cantidad_tildes)

Genero las features estáticas léxicas basadas en palabras:

In [25]:
# total number of words
def cantidad_palabras(consulta):
  palabras = consulta.split(sep=' ')
  return len(palabras)

df['cantidad_palabras'] = df['consulta'].apply(cantidad_palabras)

# proportion of short words (less than four characters)
def cantidad_palabras_cortas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return sum(1 for palabra in palabras if len(palabra) <= letras_corta)

df['cantidad_palabras_cortas'] = df['consulta'].apply(cantidad_palabras_cortas)

# ratio of number of distinct words to the total number of words: |set(words)|/|words|
def cantidad_palabras_distintas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return len(set(palabras))

df['proporcion_palabras_distintas'] = df['consulta'].apply(cantidad_palabras_distintas)/df['cantidad_palabras']
df['proporcion_palabras_distintas']

0      0.837209
1      0.885714
2      0.785714
3      0.773333
4      1.000000
         ...   
795    0.777778
796    0.892857
797    0.666667
798    0.880952
799    0.903226
Name: proporcion_palabras_distintas, Length: 800, dtype: float64

Genero las features estáticas sintácticas:

In [26]:
# Frecuencia de signos de puntuación {,.¿?!:;’"}
def cantidad_signos_puntuacion(consulta):
  signos_puntuacion = [',', '.', '¿', '?', '!', '¡', ':', ';', '"']
  return sum(1 for letra in consulta if letra.lower() in signos_puntuacion)

df['frecuencia_signos_puntuacion'] = df['consulta'].apply(cantidad_signos_puntuacion)/df['cantidad_caracteres']

Genero las features estáticas estructurales:

In [27]:
# total number of sentences
def cantidad_oraciones(consulta):
  if consulta.count('.')==0:
    return 1
  else: 
    return consulta.count('.')

df['cantidad_oraciones'] = df['consulta'].apply(cantidad_oraciones)

# utiliza código de asignatura?
def utiliza_codigo_asignatura(consulta):
  codigo=0
  palabras = consulta.split(sep=' ')
  for palabra in palabras:
    if palabra.isdigit():
      if int(palabra)>=10000 and int(palabra)<=99999:
        codigo=1
  return codigo

df['utiliza_codigo_asignatura'] = df['consulta'].apply(utiliza_codigo_asignatura)

In [28]:
len(df.columns)

25

Por último, reordeno el score y la clase para que me quede última:

In [29]:
x = df['score']
y = df['clase']
df.drop(columns=['clase', 'score'], inplace=True)
df.insert(len(df.columns), "score", x, True)
df.insert(len(df.columns), "clase", y, True)

In [30]:
# Tomo una consulta determinada
df.loc[1, 'consulta']

# Verifico las columnas del Dataframe
df.head()

,consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,score,clase
0,"hola, hice hace un mes y algo el tramite onlin...",4,1,5,1,2019,1,8,4,1,1,3,gmail,237,0.0,0.797468,1,43,25,0.837209,0.021097,2,0,37.415314,Boleto Universitario
1,"hola, en la página dice que tengo asignado el ...",1,3,10,2,2019,3,7,4,1,0,3,gmail,207,0.0,0.816425,2,35,17,0.885714,0.019324,1,0,34.819030,Boleto Universitario
2,yo hace casi un mes hice el tramite para la su...,0,5,7,1,2018,3,8,4,1,1,3,gmail,299,0.0,0.799331,0,56,33,0.785714,0.016722,3,0,33.004665,Boleto Universitario
3,"hola que tal?, buenos días. quería consultar ...",4,2,7,1,2018,0,8,4,1,1,43,gmail,425,0.0,0.800000,3,75,38,0.773333,0.025882,7,0,33.004196,Boleto Universitario
4,hola queria saber xque no puedo hacer el bolet...,1,3,5,1,2019,3,7,4,1,1,19,hotmail,114,0.0,0.815789,0,20,10,1.000000,0.017544,1,0,31.860080,Boleto Universitario


10. Guardo el csv procesado:

In [ ]:
from google.colab import drive
drive.mount('drive')

df.to_csv('dataset-tfidf-prep.csv', index=False)

!cp dataset-tfidf-prep.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
